# DEMO - Automatic Generation of Presentations from Academic Reports

Import libraries

In [ ]:
%reload_ext autoreload
%autoreload 2

import arxiv
import os
import subprocess

from dotenv import load_dotenv

from tex2beam.main import generate_presentation
from tex2beam.utils import extract_archive, folder_walker, determine_main_tex_file

load_dotenv()

## Settings

### arXiv report source
Find the report arXiv number at [arXiv](https://arxiv.org/) and insert in the cell below.

In [ ]:
arxiv_doc_id = "2406.09143"

### Number of slides for the generated presentation
Set the number of slides wanted. A title slide will be added.

In [ ]:
n_slides = 9

### Generation method
Specify the wanted generation method.  
Available methods are: `baseline`, `rag`, `two-step`, and `rag-two-step`

In [ ]:
method = 'rag'

### Download the source files from arXiv

Run the cell below to download the source file from arXiv (assuming it exists).

In [ ]:
# 1. Create tmp folder
tmp_path = ".tmp"
if not os.path.exists(tmp_path):
    os.makedirs(tmp_path)

# 2. Download source .gz file from arXiv
print("Downloading report.")
paper = next(arxiv.Client().results(arxiv.Search(id_list=[arxiv_doc_id])))
filename=f"{arxiv_doc_id}.tar.gz"
paper.download_source(dirpath=tmp_path, filename=filename)

# 3. Unpack archive
print("Extracting report.")
unpack_path = os.path.join(tmp_path, arxiv_doc_id)
extract_archive(
    filepath=os.path.join(tmp_path, filename),
    destination=unpack_path
)
print("Done")

### Generate presentation using OpenAI ChatGPT
Generate the presentation and output in $\LaTeX$ Beamer format.  

In [ ]:
# 4. Determine main .tex file
main_tex_file = determine_main_tex_file(folder_walker(unpack_path, return_files=True))

# 5. Generate presentation
print("Generating presentation. Please wait...")
presentation_path = os.path.join(os.path.dirname(main_tex_file), arxiv_doc_id + '-presentation.tex')
presentation = generate_presentation(
    report_path = main_tex_file,
    api_key = os.environ.get("OPENAI_API_KEY"),
    method = method,  # options: "baseline", "rag", "two-step", "rag-two-step"
    output_path = presentation_path,
    n_slides = n_slides
)
print("Done")

### Compile $\LaTeX$ presentation to PDF

Note that this requires a local $\LaTeX$ compiler installed on the system, e.g. [TexLive](https://tug.org/texlive/).


In [ ]:
# 6. Compile LaTex Beamer presentation into PDF
print("Compiling presentation.")
!latexmk -pdf -quiet -outdir={os.path.dirname(main_tex_file)} {presentation_path}
print("Done.")

Note that the code below assumes that this notebook is running in Windows Subsystem for Linux (WSL), using the `wslpath` command to get the full path to the generated PDF presentation.

In [ ]:
pdf_path = !wslpath -w {presentation_path.replace('.tex', '.pdf')}
subprocess.call('cmd.exe /C start'.split() + [pdf_path[0]])